### Import required libraries

In [26]:
import pandas as pd

### Set pd display options

In [27]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 10000)

### Load required data files to create initial consolidated file
#### Make a final orders df which has order_id, payment_value, order_item_id, product_id, product_category_name_english, product_weight_gms, product_length_cm, product_height_cm, product_width_cm, customer_order_id, customer_unique_id, customer_zip_code_prefix, customer_city, customer_state, customer_geolocation_lat, customer_geolocation_lng, seller_id, seller_zip_code_prefix, seller_city, seller_state, seller_geolocation_lat, seller_geolocation_lng, order_status, order_purchase_timestamp, order_approved_at, order_delivered_carrier_date, order_delivered_customer_date, order_estimated_delivery_date

In [28]:
sellers_df = pd.read_csv('../data/olist_sellers_dataset.csv')
customers_df = pd.read_csv('../data/olist_customers_dataset.csv')
geolocation_df = pd.read_csv('../data/olist_geolocation_dataset.csv', encoding='utf-8')
orders_df = pd.read_csv('../data/olist_orders_dataset.csv')
order_items_df = pd.read_csv('../data/olist_order_items_dataset.csv')
order_payments_df = pd.read_csv('../data/olist_order_payments_dataset.csv')
products_df = pd.read_csv('../data/olist_products_dataset.csv')
products_names_translation_df = pd.read_csv('../data/product_category_name_translation.csv')

### Rename/clean initial column names/dfs as required before merging
#### customer_id: customer ID token that is generated for every order. If the same customer makes multiple orders, he has multiple customer_id identifiers.  
#### customer_unique_id: which is unique to each purchaser and can be used to track their purchases over time. 
#### So, rename customer_id in orders_df and customers_df to customer_id_for_order in orders_df

In [29]:
orders_df.rename({'customer_id': 'customer_id_for_order'}, axis=1, inplace=True)
customers_df.rename({'customer_id': 'customer_id_for_order'}, axis=1, inplace=True)

### Explore the files to get basic info, stats
### Check for any missing or unusual values with any formatting errors

### utility function to explore, check, clean up files

In [30]:
def check_basic_data_issues_geo_dist_stats(df, df_type, id_type):
    
    print(f'{df_type} columns: ', list(df), '\n')
    print(f'Num of {df_type}: ', df.shape[0], '\n')

    # check if there are any missing values
    print(df.isnull().sum(), '\n') # None

    # check how many entries from each state
    print(f'{df_type} by state: \n', df[f'{df_type}_state'].value_counts(), '\n')
     
    # get the count of entries by state,city combination
    print(f'{df_type} by state and city: \n', df.groupby([f'{df_type}_state', f'{df_type}_city'])
                                                .agg({f'{id_type}':'count'})
                                                .rename(columns={f'{id_type}': f'num of {id_type}'}))
    #print(f'{df_type} by state and city: \n', df[[f'{df_type}_state', f'{df_type}_city']].value_counts(), '\n')
    # --> understand how and why this worked earlier  

    # get count of entries by zip-code prefix
   # print(f'{df_type} count by zipcode prefix: \n', df[f'{df_type}_zip_code_prefix'].value_counts(), '\n')

In [31]:
## explore, check, cleanup sellers info
## sellers df - 3095 records
check_basic_data_issues_geo_dist_stats(sellers_df, df_type='seller', id_type='seller_id')

seller columns:  ['seller_id', 'seller_zip_code_prefix', 'seller_city', 'seller_state'] 

Num of seller:  3095 

seller_id                 0
seller_zip_code_prefix    0
seller_city               0
seller_state              0
dtype: int64 

seller by state: 
 SP    1849
PR     349
MG     244
SC     190
RJ     171
RS     129
GO      40
DF      30
ES      23
BA      19
CE      13
PE       9
PB       6
MS       5
RN       5
MT       4
RO       2
SE       2
PI       1
MA       1
PA       1
AC       1
AM       1
Name: seller_state, dtype: int64 

seller by state and city: 
                                                        num of seller_id
seller_state seller_city                                               
AC           rio branco                                               1
AM           manaus                                                   1
BA           arraial d'ajuda (porto seguro)                           1
             bahia                                               

In [32]:
## explore, check, cleanup customers info
# use customer unique id since that is the unique id for a given customer
check_basic_data_issues_geo_dist_stats(customers_df, df_type='customer', id_type='customer_unique_id')

customer columns:  ['customer_id_for_order', 'customer_unique_id', 'customer_zip_code_prefix', 'customer_city', 'customer_state'] 

Num of customer:  99441 

customer_id_for_order       0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64 

customer by state: 
 SP    41746
RJ    12852
MG    11635
RS     5466
PR     5045
SC     3637
BA     3380
DF     2140
ES     2033
GO     2020
PE     1652
CE     1336
PA      975
MT      907
MA      747
MS      715
PB      536
PI      495
RN      485
AL      413
SE      350
TO      280
RO      253
AM      148
AC       81
AP       68
RR       46
Name: customer_state, dtype: int64 

customer by state and city: 
                                         num of customer_unique_id
customer_state customer_city                                     
AC             brasileia                                        1
               cruzeiro do sul                                  3
 

### Explore orders dataset

In [33]:
print('Different types of order statuses: \n', orders_df['order_status'].value_counts())

Different types of order statuses: 
 delivered      96478
shipped         1107
canceled         625
unavailable      609
invoiced         314
processing       301
created            5
approved           2
Name: order_status, dtype: int64


*Consider only 'delivered' order_status records*

In [34]:
orders_dlvrd_df = orders_df.query('order_status == "delivered"')
print('Total delivered orders: \n', orders_dlvrd_df['order_status'].value_counts())

Total delivered orders: 
 delivered    96478
Name: order_status, dtype: int64


In [35]:
# drop duplicate orders if any
orders_dlvrd_df = orders_dlvrd_df.drop_duplicates('order_id')
print('Total delivered orders without duplicates: \n', orders_dlvrd_df.shape[0])

Total delivered orders without duplicates: 
 96478


In [36]:
# check for any null or missing values
print('Missing values: \n', orders_dlvrd_df.isnull().sum(), '\n') # 24 records with atleast 1 missing value

Missing values: 
 order_id                          0
customer_id_for_order             0
order_status                      0
order_purchase_timestamp          0
order_approved_at                14
order_delivered_carrier_date      2
order_delivered_customer_date     8
order_estimated_delivery_date     0
dtype: int64 



In [37]:
# drop any missing value rows since we want only complete rows for analysis
orders_dlvrd_df.dropna(inplace=True) # 96455 records
print('Total delivered orders after dropping null/missing duplicates: \n', orders_dlvrd_df.shape[0])

Total delivered orders after dropping null/missing duplicates: 
 96455


### Get customer information into the orders dataset
#### Merge both dfs on customer_id_for_order - this should get customer_unique_id, customer_zip_code_prefix, customer_city, customer_city into the orders_df

In [139]:
## left merge on orders because we want to have all orders
orders_dlvrd_customers_df = orders_dlvrd_df.merge(customers_df, on='customer_id_for_order', how='left')

print('delivered orders with customers info: \n', orders_dlvrd_customers_df.shape[0]) # length of delivered orders df (96455)

#Now check for any missing values to see if orders and order items df have any missing values
print('orders_items_dlvrd_customers_df null/missing values: \n', orders_dlvrd_customers_df.isnull().sum())
orders_dlvrd_customers_df.to_csv('../data/orders_dlvrd_customers.csv', index=False)

delivered orders with customers info: 
 96455
orders_items_dlvrd_customers_df null/missing values: 
 order_id                         0
customer_id_for_order            0
order_status                     0
order_purchase_timestamp         0
order_approved_at                0
order_delivered_carrier_date     0
order_delivered_customer_date    0
order_estimated_delivery_date    0
customer_unique_id               0
customer_zip_code_prefix         0
customer_city                    0
customer_state                   0
dtype: int64


### Get orders geo_dist info using customer's geo dist info

In [39]:
## get orders by state
print('orders by state: \n', orders_dlvrd_customers_df['customer_state'].value_counts())

orders by state: 
 SP    40488
RJ    12348
MG    11351
RS     5342
PR     4923
SC     3546
BA     3256
DF     2080
ES     1995
GO     1957
PE     1593
CE     1278
PA      946
MT      886
MA      716
MS      701
PB      517
PI      476
RN      474
AL      397
SE      335
TO      274
RO      243
AM      145
AC       80
AP       67
RR       41
Name: customer_state, dtype: int64


In [40]:
## get orders by state,city combo
print('Orders by state and city: \n', orders_dlvrd_customers_df.groupby(['customer_state', 'customer_city'])
                                                               .agg({'order_id': 'count'})
                                                               .rename(columns={'order_id': 'num of orders'}))

Orders by state and city: 
                                         num of orders
customer_state customer_city                         
AC             brasileia                            1
               cruzeiro do sul                      3
               epitaciolandia                       1
               manoel urbano                        1
               porto acre                           1
...                                               ...
TO             silvanopolis                         1
               sitio novo do tocantins              2
               taguatinga                           3
               tocantinopolis                       7
               xambioa                              3

[4272 rows x 1 columns]


### Get sellers info into orders_dlvrd_customers_df
#### first merge orders_dlvrd_customers_df with order_items_df to get seller_id then merge with sellers_d to get sellers geo info

In [141]:
## merge orders_dlvrd_customers_df with order_items_df on 'seller_id'
# this should get order_item_id, product_id, seller_id, shipping_limit_date, price, freight_value into orders_dlvrd_customers_df
# rename price to order_item_price since the price in this df is for each item in the order

print('order_items_df: ', order_items_df.shape[0]) #112650

# left since want to retain each order even if some corresponding info from order items is missing for that order
orders_items_dlvrd_customers_df = orders_dlvrd_customers_df.merge(order_items_df, on='order_id', how='left')

print('orders_items_dlvrd_customers_df rows: ', orders_items_dlvrd_customers_df.shape[0]) # 110173 - same as order_items_df size

# Now check for any missing values to see if orders and order items df have any missing values
print('orders_items_dlvrd_customers_df null/missing values: \n', orders_items_dlvrd_customers_df.isnull().sum())
orders_items_dlvrd_customers_df.to_csv('../data/orders_items_dlvrd_customers.csv', index=False)

order_items_df:  112650
orders_items_dlvrd_customers_df rows:  110173
orders_items_dlvrd_customers_df null/missing values: 
 order_id                         0
customer_id_for_order            0
order_status                     0
order_purchase_timestamp         0
order_approved_at                0
order_delivered_carrier_date     0
order_delivered_customer_date    0
order_estimated_delivery_date    0
customer_unique_id               0
customer_zip_code_prefix         0
customer_city                    0
customer_state                   0
order_item_id                    0
product_id                       0
seller_id                        0
shipping_limit_date              0
price                            0
freight_value                    0
dtype: int64


#### now get sellers state, city, zip info from sellers_df into orders_items_dlvrd_customers_df by merging both on the basis of seller_id

In [143]:
orders_items_dlvrd_customers_sellers_df = orders_items_dlvrd_customers_df.merge(sellers_df, on='seller_id')
print(orders_items_dlvrd_customers_sellers_df.shape[0])
print(orders_items_dlvrd_customers_sellers_df.isnull().sum())
orders_items_dlvrd_customers_sellers_df.to_csv('../data/orders_items_dlvrd_customers_sellers.csv', index=False)

110173
order_id                         0
customer_id_for_order            0
order_status                     0
order_purchase_timestamp         0
order_approved_at                0
order_delivered_carrier_date     0
order_delivered_customer_date    0
order_estimated_delivery_date    0
customer_unique_id               0
customer_zip_code_prefix         0
customer_city                    0
customer_state                   0
order_item_id                    0
product_id                       0
seller_id                        0
shipping_limit_date              0
price                            0
freight_value                    0
seller_zip_code_prefix           0
seller_city                      0
seller_state                     0
dtype: int64


### Get orders fulfilled by sellers geo_dist info using sellers's geo dist info

In [43]:
# get orders fulfilled from state
orders_items_dlvrd_customers_sellers_df['seller_state'].value_counts()

SP    78585
MG     8601
PR     8485
RJ     4685
SC     3999
RS     2169
DF      883
BA      624
GO      508
PE      445
MA      402
ES      364
MT      144
CE       90
RN       56
MS       50
PB       37
RO       14
PI       11
SE       10
PA        8
AM        3
Name: seller_state, dtype: int64

In [44]:
# get orders fulfilled from state, city combo
print(orders_items_dlvrd_customers_sellers_df.groupby(['seller_state', 'seller_city'])
                                       .agg({'order_id': 'count'})
                                       .rename(columns={'order_id': 'num of orders fulfilled'}))

                                                       num of orders fulfilled
seller_state seller_city                                                      
AM           manaus                                                          3
BA           arraial d'ajuda (porto seguro)                                 28
             bahia                                                           5
             barro alto                                                      1
             eunapolis                                                      14
             feira de santana                                                3
             guanambi                                                       22
             ilheus                                                         21
             ipira                                                           1
             irece                                                           1
             lauro de freitas                       

#### Replace sao  paulo, sao paluo, sao paulo – sp, sao paulo / sao paulo, sao paulo sp, sao paulop, são paulo, sp, sp / sp with 'sao paulo' in sellers and geolocation dfs

In [45]:
pattern = '|'.join(['sao  paulo', 'sao paluo', 'sao paulo – sp', 'sao paulo / sao paulo', 'sao paulo sp',
                   'sao paulop', 'são paulo', 'sp', 'sp / sp'])

orders_items_dlvrd_customers_sellers_df['seller_city'] = orders_items_dlvrd_customers_sellers_df['seller_city'].str.replace(pattern, 'sao paulo')
orders_items_dlvrd_customers_sellers_df['seller_city'].value_counts()

sao paulo                                   27458
ibitinga                                     7617
curitiba                                     2955
santo andre                                  2886
sao jose do rio preto                        2544
belo horizonte                               2522
rio de janeiro                               2356
guarulhos                                    2308
ribeirao preto                               2208
maringa                                      2194
piracicaba                                   1893
itaquaquecetuba                              1639
campinas                                     1374
salto                                        1326
praia grande                                 1310
campo limpo paulista                         1179
guariba                                      1148
sao bernardo do campo                        1110
jacarei                                       934
limeira                                       916


### Explore, check, clean geolocation file

In [147]:
geolocation_df['geolocation_city'] = geolocation_df['geolocation_city'].str.replace(pattern, 'sao paulo')

## Get only those records in geo location file where state = 'SP' and city = 'sao paulo'
print(geolocation_df['geolocation_state'].value_counts())
print(geolocation_df['geolocation_city'].value_counts())

geolocation_sp_df = geolocation_df.query('geolocation_state == "SP" & geolocation_city == "sao paulo"')
print('geolocation_sp_df records: ', geolocation_sp_df.shape[0])
geolocation_sp_df = geolocation_sp_df.drop_duplicates(['geolocation_zip_code_prefix', 'geolocation_city', 'geolocation_state'])
print('geolocation_sp_df records after dropping duplicates: ', geolocation_sp_df.shape[0] )

geolocation_sp_df.to_csv('../data/geolocation_sp_df.csv', index=False)

SP    404268
MG    126336
RJ    121169
RS     61851
PR     57859
SC     38328
BA     36045
GO     20139
ES     16748
PE     16432
DF     12986
MT     12031
CE     11674
PA     10853
MS     10431
MA      7853
PB      5538
RN      5041
PI      4549
AL      4183
TO      3576
SE      3563
RO      3478
AM      2432
AC      1301
AP       853
RR       646
Name: geolocation_state, dtype: int64
sao paulo         160721
rio de janeiro     62151
belo horizonte     27805
curitiba           16593
porto alegre       13521
                   ...  
itainópolis            1
porto aelgre           1
cromínia               1
paraú                  1
´teresopolis           1
Name: geolocation_city, Length: 8007, dtype: int64
geolocation_sp_df records:  160719
geolocation_sp_df records after dropping duplicates:  3185


### Get the corresponding geolocation lat and long for each customer and seller in customers_df and sellers_df
#### So merge customers_df with geolocation_df and sellers_df with geolocation_df based on zip_code_prefix, city and state

#### get only sao paulo customers and sellers

In [149]:
orders_items_dlvrd_customers_sellers_sp_df = orders_items_dlvrd_customers_sellers_df.query('customer_state == "SP" & customer_city=="sao paulo" & seller_state == "SP" & seller_city=="sao paulo"')

#### Merge customer geo location info

In [151]:
print('records before cust geo merge: ', orders_items_dlvrd_customers_sellers_sp_df.shape[0])

orders_items_dlvrd_customers_sellers_cust_geo_df = orders_items_dlvrd_customers_sellers_sp_df.merge(geolocation_sp_df, 
                                                   left_on=['customer_zip_code_prefix'],
                                                right_on=['geolocation_zip_code_prefix'], how='left')

orders_items_dlvrd_customers_sellers_cust_geo_df.rename(columns={'geolocation_lat': 'customer_lat', 
                                                                 'geolocation_lng': 'customer_long'},
                                                       inplace=True)

print('records after cust geo merge: ', orders_items_dlvrd_customers_sellers_cust_geo_df.shape[0])
print(list(orders_items_dlvrd_customers_sellers_cust_geo_df))

orders_items_dlvrd_customers_sellers_cust_geo_df.to_csv('../data/orders_items_dlvrd_customers_sellers_cust_geo.csv', index=False)

records before cust geo merge:  4674
records after cust geo merge:  4674
['order_id', 'customer_id_for_order', 'order_status', 'order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date', 'customer_unique_id', 'customer_zip_code_prefix', 'customer_city', 'customer_state', 'order_item_id', 'product_id', 'seller_id', 'shipping_limit_date', 'price', 'freight_value', 'seller_zip_code_prefix', 'seller_city', 'seller_state', 'geolocation_zip_code_prefix', 'customer_lat', 'customer_long', 'geolocation_city', 'geolocation_state']


#### Merge sellers geo location info

In [155]:
print('records before sellers geo merge: ', orders_items_dlvrd_customers_sellers_cust_geo_df.shape[0])

orders_items_dlvrd_customers_sellers_cust_geo_sellers_geo_df = orders_items_dlvrd_customers_sellers_cust_geo_df.merge(geolocation_sp_df, 
                                                   left_on=['seller_zip_code_prefix'],
                                                right_on=['geolocation_zip_code_prefix'], how='left')

orders_items_dlvrd_customers_sellers_cust_geo_sellers_geo_df.rename(columns={'geolocation_lat': 'seller_lat', 
                                                                 'geolocation_lng': 'seller_long'},
                                                       inplace=True)

print('records after sellers geo merge: ', orders_items_dlvrd_customers_sellers_cust_geo_sellers_geo_df.shape[0])

orders_items_dlvrd_customers_sellers_cust_geo_sellers_geo_df.drop(['geolocation_zip_code_prefix_x', 'geolocation_zip_code_prefix_y'], axis=1, inplace=True)

# drop any nans; 10 seller zip code prefix didn'thave any geo lat long info
orders_items_dlvrd_customers_sellers_cust_geo_sellers_geo_df.dropna(inplace=True) 

# drop any city, state info since all are from SP, SP
orders_items_dlvrd_customers_sellers_cust_geo_sellers_geo_df.drop(['customer_city', 'customer_state', 'seller_city', 'seller_state',
                                                                  'geolocation_city_x', 'geolocation_state_x', 'geolocation_city_y', 'geolocation_state_y'], inplace=True, axis=1)

print(list(orders_items_dlvrd_customers_sellers_cust_geo_sellers_geo_df))


print('records after dropping nans: ', orders_items_dlvrd_customers_sellers_cust_geo_sellers_geo_df.shape[0])

orders_items_dlvrd_customers_sellers_cust_geo_sellers_geo_df.to_csv('../data/orders_items_dlvrd_customers_sellers_cust_geo_sellers_geo.csv', index=False)

records before sellers geo merge:  4674
records after sellers geo merge:  4674
['order_id', 'customer_id_for_order', 'order_status', 'order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date', 'customer_unique_id', 'customer_zip_code_prefix', 'order_item_id', 'product_id', 'seller_id', 'shipping_limit_date', 'price', 'freight_value', 'seller_zip_code_prefix', 'customer_lat', 'customer_long', 'seller_lat', 'seller_long']
records after dropping nans:  4638


### Calculate geo distance between sellers and customers

In [156]:
from geopy.distance import geodesic

orders_items_dlvrd_customers_sellers_cust_geo_sellers_geo_df['cust_lat_long'] = list(zip(orders_items_dlvrd_customers_sellers_cust_geo_sellers_geo_df['customer_lat'], 
                orders_items_dlvrd_customers_sellers_cust_geo_sellers_geo_df['customer_long']))

# print(orders_items_dlvrd_customers_sellers_cust_geo_sellers_geo_df['cust_lat_long'].head())
                                                                                         
orders_items_dlvrd_customers_sellers_cust_geo_sellers_geo_df['seller_lat_long'] = list(zip(orders_items_dlvrd_customers_sellers_cust_geo_sellers_geo_df['seller_lat'], 
                orders_items_dlvrd_customers_sellers_cust_geo_sellers_geo_df['seller_long']))

# print(orders_items_dlvrd_customers_sellers_cust_geo_sellers_geo_df['seller_lat_long'].head())

orders_items_dlvrd_customers_sellers_cust_geo_sellers_geo_df['seller_cust_geo_dist'] = orders_items_dlvrd_customers_sellers_cust_geo_sellers_geo_df.apply(lambda x: geodesic(x.seller_lat_long, x.cust_lat_long).miles, axis=1)

orders_items_dlvrd_customers_sellers_cust_geo_sellers_geo_df['seller_cust_geo_dist'].head()

op

orders_items_dlvrd_customers_sellers_cust_geo_sellers_geo_df.to_csv('../data/orders_items_cust_sellers_latlong_dist.csv', index=False)